#  Sentiment Analysis on Amazon Food Reviews

# Amazon Food Reviews Recurrent Neural Network

A recurrent neural network (RNN) is a class of artificial neural network where connections between units form a directed graph along a sequence. This allows it to exhibit dynamic temporal behavior for a time sequence. Unlike feedforward neural networks, RNNs can use their internal state (memory) to process sequences of inputs. This makes them applicable to tasks such as unsegmented, connected handwriting recognition or speech recognition

In [1]:
pwd

'/Users/user/Desktop'

# Algorithm and Deep Learning
In order to predict if a review is positive or negative we will use a recurrent neural network. Most of the methods for sentiment analysis look at each word individually, attribute positive points for positive words and negative points for negative words, and then total the points. This is the lexicon approach. However the problem with this method is that it ignores the sequence of the words, which can lead to the loss of important information. The RNN approach can understand subtleties because it doesn’t analyze text at face value. It creates abstract representations of what it learned.

We will use Long Short Term Memory Networks (LSTM), which is a special case of RNN. The main advantage of LSTM is that it can memorize information. When there's too large of a gap between two pieces of information, RNN is unable to learn to connect information.

In [56]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Input, Dense, LSTM
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
%matplotlib inline
from keras.layers.core import Flatten
from keras.layers.core import Dense

sns.set(style='whitegrid', palette='muted', font_scale=1.5)
rcParams['figure.figsize'] = 14, 8

In [57]:
# Feature Scaling
from sklearn.preprocessing import MinMaxScaler

mns = MinMaxScaler()

In [58]:
# Initialising the RNN
regressor = Sequential()

In [59]:
data = pd.read_csv("FoodReview.csv")

In [60]:
data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Prediction,Time,Summary,Text
0,492346,B001VNP0Y6,AEZRVY3CV52UZ,"Corey Wright ""Democratus""",3,3,5,1320537600,Just what I needed,I got them in a very timely manner and they're...
1,343811,B008J1HO4C,A18VZYSLOUO060,"D. Simpson ""Frugal SOB""",2,2,5,1204329600,Most excellent oatmeal,McCann's Steel Cut Oatmeal is the perfect brea...
2,347257,B000FFIL60,A28Y1M7GRG0I9M,"Real Comments ""Lin""",0,0,4,1318464000,Quality Tea,I would recommend it. Quality pearls and a few...
3,225529,B002ZX1U9A,A245HC4T5J97WG,Logan DeAngelis,0,0,5,1306540800,Great Coffee at a Great price,"I have to admit, I first purchased these Hazel..."
4,485488,B001RVFERK,AVABPJCKE2MR5,spal,0,0,4,1314921600,Popchips Jalapeno flavor,I bought a case of jalapeno chips as I love th...


In [61]:
data.columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Prediction', 'Time', 'Summary', 'Text'],
      dtype='object')

In [62]:
# split table into training table
training_table = data[['Id','HelpfulnessNumerator','HelpfulnessDenominator', 'Prediction', 'Time']]

### Output: positive or negative?
Finally, we will predict if the review is positive or negative using the sigmoid function. The advantage of the function is that it's bound between 0 and 1 and can be interpreted as a probability of success. For example, we can estimate the probability that a review is positive. At each step, we have an output. However, we only care about the final output that predicts the sentiment at the end of the sentence. 

In [63]:
training_table.head()

,Id,HelpfulnessNumerator,HelpfulnessDenominator,Prediction,Time
0,492346,3,3,5,1320537600
1,343811,2,2,5,1204329600
2,347257,0,0,4,1318464000
3,225529,0,0,5,1306540800
4,485488,0,0,4,1314921600


# Feature Scaling

In [64]:
training_table = training_table.iloc[:, 2:3].values

# Feature Scaling
from sklearn.preprocessing import MinMaxScaler
mns = MinMaxScaler()
training_table = mns.fit_transform(training_table)

/Users/user/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


### Tuning hyperparameters
For our recurrent neural network we will have to tune the following hyperparameters:

Optimizer hyperparameters:

1.) Learning rate
2.) Minibatch size
3.) Number of epochs

Model hyperparameters:

1.) Number of hidden layers (LSTM layers)

2.) Number of units in the LSTM cells

3.) Dropout rate

In [65]:
# Getting the inputs and the ouputs
x_train =  training_table[0:999]
y_train = training_table[1:1000]


In [66]:
# Reshaping
x_train = x_train.reshape(999,1,1)
print(x_train.shape)
print(y_train.shape)

(999, 1, 1)
(999, 1)


# Generate LSTM

# Long Short Term Memory
The next step is to pass the embedded words to the LSTM cells. In the graph, you can observe that there are connections between the first, second and third LSTM cells. In contrast to other models that assume that all the inputs are independent of each other, these connections here allow us to pass information contained in the sequence of words. One of the strengths of the LSTM cell is the ability to add or remove information to the cell state.

In [67]:
#LSTM
regressor.add(LSTM(12, activation='relu', input_shape=(None,1)))
regressor.add(Dense(8, activation='softmax'))
regressor.add(Dense(1))
regressor.compile(optimizer='adadelta', loss='mean_squared_error')

In [68]:
# Fitting the RNN to the Training set
regressor.fit(x_train, y_train, batch_size=20, epochs=20)

Epoch 1/20
999/999 [==============================] - 1s 885us/step - loss: 5.0509e-04
Epoch 2/20
999/999 [==============================] - 0s 489us/step - loss: 2.7751e-05
Epoch 3/20
999/999 [==============================] - 0s 392us/step - loss: 2.7957e-05
Epoch 4/20
999/999 [==============================] - 0s 428us/step - loss: 2.8000e-05
Epoch 5/20
999/999 [==============================] - 0s 411us/step - loss: 2.8153e-05
Epoch 6/20
999/999 [==============================] - 0s 414us/step - loss: 2.8068e-05
Epoch 7/20
999/999 [==============================] - 0s 411us/step - loss: 2.8173e-05
Epoch 8/20
999/999 [==============================] - 0s 420us/step - loss: 2.8092e-05
Epoch 9/20
999/999 [==============================] - 0s 422us/step - loss: 2.8661e-05
Epoch 10/20
999/999 [==============================] - 0s 421us/step - loss: 2.8412e-05
Epoch 11/20
999/999 [==============================] - 0s 421us/step - loss: 2.8253e-05
Epoch 12/20
999/999 [====================

# Simple RNN

In [69]:
from keras.layers.recurrent import SimpleRNN
from keras.layers import Dropout


To train the RRN on the review dataset we have to create an array with 200 columns, and enough rows to fit one review per row. Then store each review consisting of integers in the array. If the number of words in the review is less than 200 words, we can pad the list with extra zeros. If the number of words in the review are more than 200 words then we can limit the review to the first 200 words.

In [70]:
# Adding the input layer and the LSTM layer
model = Sequential()
model.add(SimpleRNN(128,input_shape = (None,1)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid',kernel_initializer='uniform'))
model.compile(optimizer='adam', loss='binary_crossentropy')

In [71]:
y_train = training_table[1:1000]

In [72]:
model.fit(x_train, y_train, batch_size=8, epochs=100)

Epoch 1/100
999/999 [==============================] - 1s 890us/step - loss: 0.3744
Epoch 2/100
999/999 [==============================] - 1s 541us/step - loss: 0.0416
Epoch 3/100
999/999 [==============================] - 1s 543us/step - loss: 0.0208
Epoch 4/100
999/999 [==============================] - 1s 545us/step - loss: 0.0170
Epoch 5/100
999/999 [==============================] - 1s 534us/step - loss: 0.0163
Epoch 6/100
999/999 [==============================] - 1s 540us/step - loss: 0.0159
Epoch 7/100
999/999 [==============================] - 1s 526us/step - loss: 0.0158
Epoch 8/100
999/999 [==============================] - 1s 515us/step - loss: 0.0155
Epoch 9/100
999/999 [==============================] - 1s 560us/step - loss: 0.0157
Epoch 10/100
999/999 [==============================] - 1s 544us/step - loss: 0.0158
Epoch 11/100
999/999 [==============================] - 1s 565us/step - loss: 0.0156
Epoch 12/100
999/999 [==============================] - 1s 701us/step - lo

In [73]:
# evaluate the model
scores = model.evaluate(x_train, y_train)
print("\n%s: %.2f%%" % (model.metrics_names, scores*100))

999/999 [==============================] - 0s 211us/step

['loss']: 1.53%


# Improvement
Deep Learning techniques tend to outperform every other algorithm when we get access to lots of data. So if we could train our model on a larger dataset, the model should definitely improve. With RNN, the model creates it's own representation of the sentence. The reviews contain vocabulary specific to the food product. If we had mode data, our model should be able to identify the specific characteristics that make a good product. In order to improve this model, we should also investigate if we could include the product name/product type. If we had this information maybe our RNN would be able to more easily identify the important characteristics for each product.

# REFERENCES

1.) Burke, R., Hybrid Recommender Systems: Survey and Experiments, <http://josquin.cs.depaul.edu/rburke/pubs/burke-umuai02.pdf>.

2.) Gunawardana A., Shani G, Evaluating Recommendation Systems, <http://research.microsoft.com/pubs/115396/evaluationmetrics.tr.pdf>.

3.) Wikipedia, Singular Value Decomposition, <http://en.wikipedia.org/ wiki/Singular value decomposition>.

4.) Koren, Y., Factorization Meets the Neighborhood: a Multifaceted Col-laborative Filtering Model,<http://public.research.att.com/ volinsky/netflix/kdd08koren.pdf>.

5.) Shang, M., Fu, Y., Chen, D., Personal Recommendation Using Weighted BiPartite Graph Projection, Apperceiving Computing and Intelligence Analysis, 2008. ICACIA 2008. International Conference on , vol., no., pp.198,202,
13-15 Dec. 2008

6.) Breese, J.S., Heckerman, D., Kadie, C., Empirical Analysis of Predictive Algorithms for Collaborative Filtering, <http://research.microsoft.com/ pubs/69656/tr-98-12.pdf>


# LICENSE : 

The code in the document by Ronak Shingala and Spurthi Shetty is licensed under the MIT License https://opensource.org/licenses/MIT

https://github.com/shingalaronak/Advance-Data-Science-Final-Project